In [38]:
import pandas as pd
from IPython.display import display, HTML

traditional_csvFile = "csv/compressed_traditional_CSV.csv"
vpTree_csvFile = "csv/compressed_VPtree_CSV.csv"
enhanced_csvFile = "csv/compressed_enhanced_CSV.csv"
cnnModel_csvFile = "csv/trained_cnn_CSV.csv"

In [39]:
# CNN MODEL TRAINING TIME & ACCURACY
columns_to_display = ["Epoch", 
                      "Batch", 
                      "Training Loss", 
                      "Training Time (s)"]

try:
    # Read CSV files
    df_cnn = pd.read_csv(cnnModel_csvFile)[columns_to_display]

    # Compute the average Training Loss and total Training Time per epoch
    df_summary = df_cnn.groupby("Epoch").agg({
        "Training Loss": "mean",  # Average training loss per epoch
        "Training Time (s)": "sum"  # Total training time per epoch
    }).reset_index()

    # Display the summarized data
    display(df_summary)

except FileNotFoundError as e:
    print(f"Error: {e}")

,Epoch,Training Loss,Training Time (s)
0,1,4.448427,79.0690
1,2,3.447695,81.8670
2,3,3.366121,83.0977
3,4,3.301909,84.4567
4,5,3.264943,82.7571
5,6,3.242062,79.0781
6,7,3.226956,78.7745
7,8,3.216202,78.7636
8,9,3.211485,78.6790
9,10,3.206481,78.4997


In [40]:
# PROPOSED KD-TREE + CNN COMPRESSION TIME ANALYSIS
columns_to_display = ["Original Image", 
                      "Build Tree Time (ms)", 
                      "Nearest Search Time (ms)", 
                      "CNN Inference Time (ms)"]

try:
    # Read CSV files
    df_enhanced = pd.read_csv(enhanced_csvFile)[columns_to_display]

    # Rename columns for clarity
    df_enhanced = df_enhanced.rename(columns={"Original Image": "Image Names"})

    ordered_columns = [
        "Image Names",
        "Build Tree Time (ms)", 
        "Nearest Search Time (ms)", 
        "CNN Inference Time (ms)"
    ]

    # Display the first 5 rows
    display(df_enhanced.head(5))

except FileNotFoundError as e:
    print(f"Error: {e}")

,Image Names,Build Tree Time (ms),Nearest Search Time (ms),CNN Inference Time (ms)
0,glioma_0001.jpg,6.5148,0.6450,172.1623
1,glioma_0002.jpg,5.9116,0.6911,49.8588
2,glioma_0003.jpg,6.6431,0.6275,16.4592
3,glioma_0004.jpg,4.3349,0.4490,17.3173
4,glioma_0005.jpg,6.3908,0.5327,16.1941


In [41]:
# COMPARISON | traditional vs proposed | compression time and ratio
# Columns to compare
columns_to_compare = ["Original Image", 
                      "Encoding Time (s)", 
                      "Decoding Time (s)", 
                      "Original Image Size (KB)", 
                      "Compressed Image Size (KB)", 
                      "Compression Ratio"]

try:
    # Read CSV files
    df_traditional = pd.read_csv(traditional_csvFile)[columns_to_compare]
    df_enhanced = pd.read_csv(enhanced_csvFile)[columns_to_compare]

    # Rename columns for clarity
    df_traditional = df_traditional.rename(columns=lambda x: f"Traditional {x}" if x != "Original Image" else x)
    df_enhanced = df_enhanced.rename(columns=lambda x: f"Enhanced {x}" if x != "Original Image" else x)

    # Merge data on "Original Image"
    comparison_df = pd.merge(df_traditional, df_enhanced, on="Original Image")
    comparison_df = comparison_df.rename(columns={"Original Image": "Image Names"})

    # Reorder columns for side-by-side comparison
    ordered_columns = [
        "Image Names",
        "Traditional Encoding Time (s)", "Enhanced Encoding Time (s)",
        "Traditional Decoding Time (s)", "Enhanced Decoding Time (s)",
        "Traditional Original Image Size (KB)", "Enhanced Original Image Size (KB)",
        "Traditional Compressed Image Size (KB)", "Enhanced Compressed Image Size (KB)",
        "Traditional Compression Ratio", "Enhanced Compression Ratio"
    ]
    comparison_df = comparison_df[ordered_columns]

    # Display the first 5 rows
    display(comparison_df.head(5))

except FileNotFoundError as e:
    print(f"Error: {e}")


,Image Names,Traditional Encoding Time (s),Enhanced Encoding Time (s),Traditional Decoding Time (s),Enhanced Decoding Time (s),Traditional Original Image Size (KB),Enhanced Original Image Size (KB),Traditional Compressed Image Size (KB),Enhanced Compressed Image Size (KB),Traditional Compression Ratio,Enhanced Compression Ratio
0,glioma_0001.jpg,107.9857,0.8457,0.1107,0.0676,40.51,40.51,7.70,7.34,1:5,1:5
1,glioma_0002.jpg,114.4855,0.7641,0.0158,0.0354,45.25,45.25,8.53,8.10,1:5,1:5
2,glioma_0003.jpg,113.5626,0.6732,0.0158,0.0365,34.63,34.63,6.52,6.19,1:5,1:5
3,glioma_0004.jpg,118.0764,0.4882,0.0000,0.0345,32.02,32.02,5.96,5.70,1:5,1:5
4,glioma_0005.jpg,118.3600,0.5764,0.0075,0.0467,28.20,28.20,5.33,5.04,1:5,1:5


In [42]:
# COMPARISON | traditional vs proposed | image quality
# Columns to compare
columns_to_compare = ["Original Image", 
                      "Original Image Path", 
                      "Compressed Image Path", 
                      "PSNR (dB)", 
                      "SSIM"]

try:
    # Read CSV files
    df_traditional = pd.read_csv(traditional_csvFile)[columns_to_compare]
    df_enhanced = pd.read_csv(enhanced_csvFile)[columns_to_compare]

    # Rename columns for clarity
    df_traditional = df_traditional.rename(columns=lambda x: f"Traditional {x}" if x != "Original Image" else x)
    df_enhanced = df_enhanced.rename(columns=lambda x: f"Enhanced {x}" if x != "Original Image" else x)

    # Merge data on "Original Image"
    comparison_df = pd.merge(df_traditional, df_enhanced, on="Original Image")

    # Remove "data/" prefix from paths
    for col in ["Traditional Original Image Path", "Traditional Compressed Image Path", "Enhanced Compressed Image Path"]:
        comparison_df[col] = comparison_df[col].str.replace(r"^data/", "", regex=True)

    # Function to format Image Name + Image in one column
    def format_image_name(image_name, path):
        return f'<div style="text-align: center;"><b>{image_name}</b><br><img src="{path}" width="100"/></div>'

    # Apply the function to combine "Image Names" and "Original Image"
    comparison_df["Image Info"] = comparison_df.apply(lambda row: format_image_name(row["Original Image"], row["Traditional Original Image Path"]), axis=1)

    # Convert compressed image paths into images
    def path_to_img_html(path):
        return f'<img src="{path}" width="100"/>'

    comparison_df["Traditional Compressed Image"] = comparison_df["Traditional Compressed Image Path"].apply(path_to_img_html)
    comparison_df["Enhanced Compressed Image"] = comparison_df["Enhanced Compressed Image Path"].apply(path_to_img_html)

    # Reorder columns for side-by-side comparison
    ordered_columns = [
        "Image Info",  # Combined "Image Names" and "Original Image"
        "Traditional Compressed Image", "Enhanced Compressed Image",
        "Traditional PSNR (dB)", "Enhanced PSNR (dB)",
        "Traditional SSIM", "Enhanced SSIM"
    ]
    comparison_df = comparison_df[ordered_columns]

    # Display DataFrame as HTML with images
    display(HTML(comparison_df.head(5).to_html(escape=False)))

except FileNotFoundError as e:
    print(f"Error: {e}")

,Image Info,Traditional Compressed Image,Enhanced Compressed Image,Traditional PSNR (dB),Enhanced PSNR (dB),Traditional SSIM,Enhanced SSIM
0,glioma_0001.jpg,,,39.3766,24.0305,0.9752,0.8260
1,glioma_0002.jpg,,,38.6396,23.9329,0.9727,0.8028
2,glioma_0003.jpg,,,40.8086,28.0477,0.9780,0.8794
3,glioma_0004.jpg,,,40.1508,25.2129,0.9801,0.8779
4,glioma_0005.jpg,,,42.0036,27.9213,0.9825,0.8966


In [43]:
# COMPARISON | vp-tree vs proposed | compression time and ratio
# Columns to compare
columns_to_compare = ["Original Image", 
                      "Encoding Time (s)", 
                      "Decoding Time (s)", 
                      "Original Image Size (KB)", 
                      "Compressed Image Size (KB)", 
                      "Compression Ratio"]

try:
    # Read CSV files
    df_VPtree = pd.read_csv(vpTree_csvFile)[columns_to_compare]
    df_enhanced = pd.read_csv(enhanced_csvFile)[columns_to_compare]

    # Rename columns for clarity
    df_VPtree = df_VPtree.rename(columns=lambda x: f"VP-tree {x}" if x != "Original Image" else x)
    df_enhanced = df_enhanced.rename(columns=lambda x: f"Enhanced {x}" if x != "Original Image" else x)

    # Merge data on "Original Image"
    comparison_df = pd.merge(df_VPtree, df_enhanced, on="Original Image")
    comparison_df = comparison_df.rename(columns={"Original Image": "Image Names"})

    # Reorder columns for side-by-side comparison
    ordered_columns = [
        "Image Names",
        "VP-tree Encoding Time (s)", "Enhanced Encoding Time (s)",
        "VP-tree Decoding Time (s)", "Enhanced Decoding Time (s)",
        "VP-tree Original Image Size (KB)", "Enhanced Original Image Size (KB)",
        "VP-tree Compressed Image Size (KB)", "Enhanced Compressed Image Size (KB)",
        "VP-tree Compression Ratio", "Enhanced Compression Ratio"
    ]
    comparison_df = comparison_df[ordered_columns]

    # Display the first 5 rows
    display(comparison_df.head(5))

except FileNotFoundError as e:
    print(f"Error: {e}")

,Image Names,VP-tree Encoding Time (s),Enhanced Encoding Time (s),VP-tree Decoding Time (s),Enhanced Decoding Time (s),VP-tree Original Image Size (KB),Enhanced Original Image Size (KB),VP-tree Compressed Image Size (KB),Enhanced Compressed Image Size (KB),VP-tree Compression Ratio,Enhanced Compression Ratio
0,glioma_0001.jpg,0.5130,0.8457,0.0825,0.0676,40.51,40.51,7.08,7.34,1:5,1:5
1,glioma_0002.jpg,0.1351,0.7641,0.0369,0.0354,45.25,45.25,7.79,8.10,1:5,1:5
2,glioma_0003.jpg,0.0752,0.6732,0.0402,0.0365,34.63,34.63,6.02,6.19,1:5,1:5
3,glioma_0004.jpg,0.2569,0.4882,0.0369,0.0345,32.02,32.02,5.53,5.70,1:5,1:5
4,glioma_0005.jpg,0.2244,0.5764,0.0333,0.0467,28.20,28.20,4.87,5.04,1:5,1:5


In [44]:
# COMPARISON | vp-tree vs proposed | image quality
# Columns to compare
columns_to_compare = ["Original Image", 
                      "Original Image Path", 
                      "Compressed Image Path", 
                      "PSNR (dB)", 
                      "SSIM"]

try:
    # Read CSV files
    df_VPtree = pd.read_csv(vpTree_csvFile)[columns_to_compare]
    df_enhanced = pd.read_csv(enhanced_csvFile)[columns_to_compare]

    # Rename columns for clarity
    df_VPtree = df_VPtree.rename(columns=lambda x: f"VP-tree {x}" if x != "Original Image" else x)
    df_enhanced = df_enhanced.rename(columns=lambda x: f"Enhanced {x}" if x != "Original Image" else x)

    # Merge data on "Original Image"
    comparison_df = pd.merge(df_VPtree, df_enhanced, on="Original Image")

    # Remove "data/" prefix from paths
    for col in ["VP-tree Original Image Path", "VP-tree Compressed Image Path", "Enhanced Compressed Image Path"]:
        comparison_df[col] = comparison_df[col].str.replace(r"^data/", "", regex=True)

    # Function to format Image Name + Image in one column
    def format_image_name(image_name, path):
        return f'<div style="text-align: center;"><b>{image_name}</b><br><img src="{path}" width="100"/></div>'

    # Apply the function to combine "Image Names" and "Original Image"
    comparison_df["Image Info"] = comparison_df.apply(lambda row: format_image_name(row["Original Image"], row["VP-tree Original Image Path"]), axis=1)

    # Convert compressed image paths into images
    def path_to_img_html(path):
        return f'<img src="{path}" width="100"/>'

    comparison_df["VP-tree Compressed Image"] = comparison_df["VP-tree Compressed Image Path"].apply(path_to_img_html)
    comparison_df["Enhanced Compressed Image"] = comparison_df["Enhanced Compressed Image Path"].apply(path_to_img_html)

    # Reorder columns for side-by-side comparison
    ordered_columns = [
        "Image Info",  # Combined "Image Names" and "Original Image"
        "VP-tree Compressed Image", "Enhanced Compressed Image",
        "VP-tree PSNR (dB)", "Enhanced PSNR (dB)",
        "VP-tree SSIM", "Enhanced SSIM"
    ]
    comparison_df = comparison_df[ordered_columns]

    # Display DataFrame as HTML with images
    display(HTML(comparison_df.head(5).to_html(escape=False)))

except FileNotFoundError as e:
    print(f"Error: {e}")

,Image Info,VP-tree Compressed Image,Enhanced Compressed Image,VP-tree PSNR (dB),Enhanced PSNR (dB),VP-tree SSIM,Enhanced SSIM
0,glioma_0001.jpg,,,22.2738,24.0305,0.7402,0.8260
1,glioma_0002.jpg,,,22.1258,23.9329,0.7152,0.8028
2,glioma_0003.jpg,,,25.5350,28.0477,0.7995,0.8794
3,glioma_0004.jpg,,,23.6932,25.2129,0.8238,0.8779
4,glioma_0005.jpg,,,25.7694,27.9213,0.8344,0.8966
